# Using GutenbergReader for Accessing Project Gutenberg Books
In this notebook, we will demonstrate how to use the `GutenbergReader` class for accessing books from Project Gutenberg. The `GutenbergReader` class interacts with the Project Gutenberg website to retrieve book content and metadata, which can be valuable for various applications, including research and question-answering systems.

To get started, ensure you have your Project Gutenberg book IDs ready for fetching the content. This setup is crucial for accessing and processing book data effectively.

To begin, ensure you have set up your environment variables and API keys in Python using the dotenv library. This is crucial for securely managing sensitive information, such as API keys, especially when using services like HuggingFace. Ensure your `HUGGINGFACE_API_KEY` is defined in the `.env` file to avoid hardcoding sensitive data into your codebase, thus enhancing security and maintainability.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/osllmai/inDox/blob/master/Demo/guten_chroma_semantic.ipynb)

In [ ]:
!pip install indox
!pip install chromadb
!pip install beautifulsoup4

## Setting Up the Python Environment

If you are running this project in your local IDE, please create a Python environment to ensure all dependencies are correctly managed. You can follow the steps below to set up a virtual environment named `indox`:

### Windows

1. **Create the virtual environment:**
```bash
python -m venv indox
```
2. **Activate the virtual environment:**
```bash
indox_judge\Scripts\activate
```

### macOS/Linux

1. **Create the virtual environment:**
   ```bash
   python3 -m venv indox
    ```

2. **Activate the virtual environment:**
    ```bash
   source indox/bin/activate
    ```
### Install Dependencies

Once the virtual environment is activated, install the required dependencies by running:

```bash
pip install -r requirements.txt
```


## Import Essential Libraries

Next, we import the essential libraries for our Indox question-answering system:

- `IndoxRetrievalAugmentation`: Enhances the retrieval process by improving the relevance and quality of the documents retrieved, leading to better QA performance.
- `MistralQA`: A powerful QA model provided by Indox, built on top of the Hugging Face model architecture. It leverages state-of-the-art language understanding to deliver precise answers.
- `HuggingFaceEmbedding`: This library uses Hugging Face embeddings to enrich semantic understanding, making it easier to capture the contextual meaning of the text.
- `SemanticTextSplitter`: utilizes a Hugging Face tokenizer to intelligently split text into chunks based on a specified maximum number of tokens, ensuring that each chunk maintains semantic coherence.

In [6]:
import os
from dotenv import load_dotenv

load_dotenv('api.env')

HUGGINGFACE_API_KEY = os.environ['HUGGINGFACE_API_KEY']

In [7]:
from indox import IndoxRetrievalAugmentation
indox = IndoxRetrievalAugmentation()

INFO: IndoxRetrievalAugmentation initialized

            ██  ███    ██  ██████   ██████  ██       ██
            ██  ████   ██  ██   ██ ██    ██   ██  ██
            ██  ██ ██  ██  ██   ██ ██    ██     ██
            ██  ██  ██ ██  ██   ██ ██    ██   ██   ██
            ██  ██  █████  ██████   ██████  ██       ██
            


## Building the GutenbergReader System and Initializing Models
Next, we will build our `GutenbergReader` system and initialize the necessary models for processing content from Project Gutenberg. This setup will enable us to effectively retrieve and handle texts from Gutenberg's collection, leveraging these models to support various research and question-answering tasks.

In [8]:
from indox.llms import HuggingFaceModel
from indox.embeddings import HuggingFaceEmbedding


mistral_qa = HuggingFaceModel(api_key=HUGGINGFACE_API_KEY,model="mistralai/Mistral-7B-Instruct-v0.2")
embed = HuggingFaceEmbedding(api_key=HUGGINGFACE_API_KEY,model="multi-qa-mpnet-base-cos-v1")

INFO: Initializing HuggingFaceModel with model: mistralai/Mistral-7B-Instruct-v0.2
INFO: HuggingFaceModel initialized successfully
INFO: Initialized HuggingFaceEmbedding with model: multi-qa-mpnet-base-cos-v1


## Setting Up the GutenbergReader for Retrieving Book Content
To demonstrate the capabilities of our `GutenbergReader` system and its integration with `Indox`, we will use a sample book from Project Gutenberg. This book will serve as our reference data, which we will use for testing and evaluation of the system.

In [21]:
from indox.data_connector import GutenbergReader

reader = GutenbergReader()

book_id = "11"  # Alice's Adventures in Wonderland
content = reader.get_content(book_id)

## Splitting Content into Manageable Chunks
We use the `SemanticTextSplitter` function from the `indox.splitter` module to divide the retrieved content into smaller, meaningful chunks.

In [22]:
from indox.splitter import SemanticTextSplitter
splitter = SemanticTextSplitter(400)
content_chunks = splitter.split_text(content)

## Storing and Indexing Content with Chroma
We use the `Chroma` vector store from the `indox.vector_stores` module to store and index the content chunks. By creating a collection named "sample" and applying an embedding function (`embed`), we convert each chunk into a vector for efficient retrieval. The `add` method then adds these vectors to the database, enabling scalable and effective search for question-answering tasks.

In [23]:
from indox.vector_stores import Chroma
db = Chroma(collection_name="sample",embedding_function=embed)
db.add(docs=content_chunks)


INFO: Storing documents in the vector store
INFO: Embedding documents
INFO: Starting to fetch embeddings for texts using model: SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)
INFO: Document added successfully to the vector store.
INFO: Documents stored successfully


## Querying Project Gutenberg Data with Indox
With our `GutenbergReader` system and `Indox` fully set up, we are ready to test it using a sample query. This test will demonstrate how effectively our system can retrieve and process information from books in the Project Gutenberg collection.

We’ll use the following sample query to evaluate our system:

- Query: "Who is the speaker talking to in the text?"

This question will be processed by the `GutenbergReader` and `Indox` system to retrieve relevant book content and generate an accurate response based on the text.

Let’s put our setup to the test with this query.

In [24]:
query = "Who is the speaker talking to in the text?"
retriever = indox.QuestionAnswer(vector_database=db, llm=mistral_qa, top_k=1)

Now that our `GutenbergReader` system with `Indox` is fully set up, we can test it with a sample query. We’ll use the invoke method to get a response from the system.

The `invoke` method processes the query using the connected QA model and retrieves relevant information from the book content.

We’ll pass the query to the `invoke` method and print the response to evaluate how effectively the system retrieves and generates answers based on the Project Gutenberg content.

In [25]:
answer = retriever.invoke(query)
context = retriever.context

INFO: Retrieving context and scores from the vector database
INFO: Embedding documents
INFO: Starting to fetch embeddings for texts using model: SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)
INFO: Generating answer without document relevancy filter
INFO: Answering question
INFO: Sending request to Hugging Face API
INFO: Received successful response from Hugging Face API
INFO: Query answered successfully


In [26]:
answer

'The speaker is Alice, and she is talking to the crowd of animals that has gathered around Bill the Rabbit.'